# Dataframe filter
> Simple dataframe filter, Interactivel y filter column by value, one by one

In [1]:
from forgebox.imports import *

HTML(value='')

In [2]:
def get_cal_housing():
    from sklearn.datasets import california_housing
    data = california_housing.fetch_california_housing()
    df = pd.DataFrame(data['data'], columns=data['feature_names'])
    return df

In [3]:
df = get_cal_housing()

/Users/salvor/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [4]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [5]:
"32.8%">"35.2%", "32.8%">"15.2%"

(False, True)

In [6]:
df.head(2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.97188,2401.0,2.109842,37.86,-122.22


## A tool to filter data

In [7]:
from forgebox.filter_df import DataFilter

In [8]:
data_filter = DataFilter(df)

There should be interactive widget at this step

In [ ]:
data_filter(columns=["AveBedrms", "Latitude", "TestPercent"])

## Extract out the dataframe afterwards

In [11]:
data_filter.df.head(1)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23


## RecursiveFilter

In [12]:
from forgebox.filter_df import RecursiveFilter

In [13]:
df = RecursiveFilter(df)

There should be interactive widget at this step

In [ ]:
df.handpick()